In [1]:
import numpy as np
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import itertools
from scipy import optimize
from sklearn.metrics import r2_score
from scipy.stats import t

<ipython-input-1-b5caf3c40180>:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Create your connection.
cnx = sqlite3.connect("data/Marketing_DS_task_2.db")

revenue_data = pd.read_sql_query("SELECT * FROM revenue_data", cnx)
spend_data = pd.read_sql_query("SELECT * FROM spend_data", cnx)

In [3]:
spend_data_country_group = (
    spend_data.groupby(
        [
            "platform",
            "country_group",
            "channel",
            "media_type",
            "install_month",
        ]
    )
    .sum("spend")
    .reset_index()
)

In [4]:
spend_revenue_df = pd.merge(
    spend_data_country_group,
    revenue_data,
    how="inner",
    on=["platform", "country_group", "channel", "media_type", "install_month"],
)
spend_revenue_df.sort_values(
    by=[
        "install_month",
        "platform",
        "country_group",
        "channel",
        "media_type",
    ],
    inplace=True,
)

In [5]:
spend_revenue_df.replace("", np.nan, inplace=True)

# Check if 'Revenue' is in column names
for col in spend_revenue_df.columns:
    if "Revenue" in col:
        # Convert 'Revenue' column to float

        spend_revenue_df[col] = spend_revenue_df[col].astype(float)

spend_revenue_df["spend"] = spend_revenue_df["spend"].astype(float)
spend_revenue_df["installs"] = spend_revenue_df["installs"].astype(int)
spend_revenue_df["cpi"] = spend_revenue_df["spend"] / spend_revenue_df["installs"]
spend_revenue_df["install_month"] = pd.to_datetime(
    spend_revenue_df["install_month"], format="%Y-%m-%d"
)

<ipython-input-5-9c42d1fe4a09>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  spend_revenue_df.replace("", np.nan, inplace=True)


In [6]:
# mask na values or 0 spend values
spend_revenue_df = spend_revenue_df[
    (spend_revenue_df["installs"] > 0)
    & (spend_revenue_df["spend"] > 0)
    # & (spend_revenue_df["D30_Revenue"].notna())
    # & (spend_revenue_df["D120_Revenue"].notna())
]
spend_revenue_df.dropna(inplace=True)

In [7]:
# Check if 'Revenue' is in column names
for col in spend_revenue_df.columns:
    if "Revenue" in col:
        # Convert 'Revenue' column to float
        period = col.split("_")[0]
        spend_revenue_df[f"{period}_arpi"] = (
            spend_revenue_df[col] / spend_revenue_df["installs"]
        )
        spend_revenue_df.drop(columns=[col], inplace=True)

spend_revenue_df.head(60)

,platform,country_group,channel,media_type,install_month,spend,installs,cpi,D1_arpi,D7_arpi,D14_arpi,D21_arpi,D30_arpi,D60_arpi,D90_arpi,D120_arpi,D150_arpi,D180_arpi
0,android,Group_1,Networks,Demand-Side-Platform,2022-01-01,71400.67,12706,5.619445,0.307365,0.789110,1.175714,1.479605,1.871305,2.933325,3.550394,3.974672,4.385082,3.974672
22,android,Group_1,Networks,Incentivized,2022-01-01,26427.15,21404,1.234683,0.235817,0.608880,0.869778,1.027187,1.145385,1.523574,1.809721,1.916510,1.981350,1.916510
44,android,Group_1,Networks,Pre-installed,2022-01-01,7563.02,4866,1.554258,0.094447,0.439166,0.634457,0.891270,1.150927,2.093726,2.863681,3.130423,3.252392,3.130423
67,android,Group_1,Networks,Video Networks,2022-01-01,6284.33,2582,2.433900,0.037184,0.111038,0.132405,0.135302,0.137405,0.142665,0.144241,0.150290,0.151340,0.150290
89,android,Group_1,Search,Search,2022-01-01,121033.79,19564,6.186556,0.065242,0.149986,0.226388,0.294749,0.373888,0.563881,0.695946,0.804624,0.879140,0.934009
111,android,Group_1,Social,Social,2022-01-01,13526.48,2978,4.542136,0.081437,0.167246,0.212760,0.255702,0.293183,0.375484,0.395406,0.461293,0.547085,0.547085
133,android,Group_2,Networks,Demand-Side-Platform,2022-01-01,19311.52,5950,3.245634,0.196718,0.371607,0.488635,0.565052,0.620509,0.895094,1.063188,1.163518,1.249499,1.163518
155,android,Group_2,Networks,Incentivized,2022-01-01,23608.26,8060,2.929065,0.396866,0.737143,0.886333,0.977524,1.070172,1.226228,1.278187,1.320700,1.376752,1.320700
177,android,Group_2,Networks,Pre-installed,2022-01-01,1048.26,240,4.367750,0.000000,0.000000,0.005625,0.048125,0.048125,0.050917,0.110458,0.130250,0.368708,0.130250
196,android,Group_2,Networks,Video Networks,2022-01-01,6047.21,2115,2.859201,0.120383,0.218804,0.337593,0.394884,0.515064,0.841035,1.198704,1.465390,1.740983,1.465390


In [8]:
spend_revenue_df.drop(["install_month"], inplace=True, axis=1)

spend_revenue_df = (
    spend_revenue_df.groupby(["platform", "country_group", "channel", "media_type"])
    .mean()
    .reset_index()
)

In [9]:
data_ltv = spend_revenue_df.copy()


data_ltv.set_index(
    [
        "platform",
        "country_group",
        "channel",
        "media_type",
        # "quarter_of_year",
    ],
    inplace=True,
)


data_ltv.drop(columns=["spend", "installs", "cpi"], inplace=True)
data_ltv = data_ltv.stack().reset_index().rename(columns={"level_4": "age", 0: "arpi"})
data_ltv["age"] = data_ltv["age"].str.replace("_arpi", "")
data_ltv["age"] = data_ltv["age"].str.replace("D", "").astype(int)

In [10]:
data_cpi = spend_revenue_df.copy()

data_cpi.set_index(
    [
        "platform",
        "country_group",
        "channel",
        "media_type",
    ],
    inplace=True,
)

data_cpi = (
    data_cpi.groupby(["platform", "country_group", "channel", "media_type"])["cpi"]
    .mean()
    .reset_index()
)
data_cpi.head(60)

,platform,country_group,channel,media_type,cpi
0,android,Group_1,Networks,Demand-Side-Platform,6.011960
1,android,Group_1,Networks,Incentivized,5.738037
2,android,Group_1,Networks,Pre-installed,2.722572
3,android,Group_1,Networks,Video Networks,4.130682
4,android,Group_1,Search,Search,6.758748
5,android,Group_1,Social,Social,6.505469
6,android,Group_2,Networks,Demand-Side-Platform,2.203641
7,android,Group_2,Networks,Incentivized,2.549163
8,android,Group_2,Networks,Pre-installed,3.214532
9,android,Group_2,Networks,Video Networks,1.991001


In [11]:
data_installs = spend_revenue_df.copy()

data_installs.set_index(
    [
        "platform",
        "country_group",
        "channel",
        "media_type",
    ],
    inplace=True,
)

data_installs = (
    data_installs.groupby(["platform", "country_group", "channel", "media_type"])[
        ["spend", "installs"]
    ]
    .mean()
    .reset_index()
)
data_installs.head(60)

,platform,country_group,channel,media_type,spend,installs
0,android,Group_1,Networks,Demand-Side-Platform,56324.191875,9726.437500
1,android,Group_1,Networks,Incentivized,24807.667500,10321.437500
2,android,Group_1,Networks,Pre-installed,18979.160000,6132.062500
3,android,Group_1,Networks,Video Networks,4237.090000,1621.142857
4,android,Group_1,Search,Search,98602.936250,14130.812500
5,android,Group_1,Social,Social,7937.026875,1286.437500
6,android,Group_2,Networks,Demand-Side-Platform,9687.618125,4400.187500
7,android,Group_2,Networks,Incentivized,9338.300000,2823.153846
8,android,Group_2,Networks,Pre-installed,1674.663333,733.500000
9,android,Group_2,Networks,Video Networks,2854.410000,1235.166667


In [12]:
data = pd.merge(
    data_ltv,
    data_cpi,
    how="inner",
    on=["platform", "country_group", "channel", "media_type"],
)

data = pd.merge(
    data,
    data_installs,
    how="inner",
    on=["platform", "country_group", "channel", "media_type"],
)

In [13]:
data["arpi/cac"] = data["arpi"] / data["cpi"]

data["break_even_age"] = np.where(data["arpi/cac"] > 1, data["age"], np.nan)
data.dropna(inplace=True)

data.sort_values(
    by=["break_even_age", "arpi/cac"], ascending=[True, False], inplace=True
)

data.drop("age", axis=1, inplace=True)
data.drop_duplicates(
    subset=["platform", "country_group", "channel", "media_type"],
    keep="first",
    inplace=True,
)
data

,platform,country_group,channel,media_type,arpi,cpi,spend,installs,arpi/cac,break_even_age
185,ios,Group_1,Networks,Incentivized,1.452266,1.108683,3587.475833,2612.250000,1.309901,60.0
175,ios,Group_1,Networks,Demand-Side-Platform,13.577828,10.885054,89977.507500,7499.437500,1.247383,60.0
305,ios,Group_3,Search,Search,3.012496,2.877556,1658.906875,589.250000,1.046894,60.0
225,ios,Group_2,Networks,Demand-Side-Platform,8.291101,7.981827,19382.248000,2295.933333,1.038747,60.0
6,android,Group_1,Networks,Demand-Side-Platform,6.303583,6.011960,56324.191875,9726.437500,1.048507,90.0
67,android,Group_2,Networks,Demand-Side-Platform,2.254455,2.203641,9687.618125,4400.187500,1.023059,120.0


In [ ]:
data.rename(
    columns= {"arpi":'Avg. Revenue per install',"spend":'Avg. Spend',"cpi":'Avg. Cost per install',"installs":'Avg. Number of installs',"break_even_age":'Avg. Break-even at, days'}, inplace=True)
data